In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../../data/processed/features.csv")
df = df.drop(columns=['Unnamed: 0'])
df.head(5)

,amount,past_transaction_count,user_avg_amount,amount_minus_user_avg,time_gap_minutes,hour,day_of_week
0,2258,0,0.0,2258.0,0.0,6,2
1,4884,0,0.0,4884.0,0.0,12,2
2,5761,0,0.0,5761.0,0.0,17,2
3,2080,0,0.0,2080.0,0.0,21,2
4,499,1,2258.0,-1759.0,1633.0,10,3


In [4]:
X = df
from sklearn.model_selection import train_test_split

X_train, X_test= train_test_split(X, test_size=0.2,random_state=42 )

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train) # Computes mean and STD of each column

,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True


In [6]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Training the model
from sklearn.ensemble import IsolationForest
model = IsolationForest(random_state=42, contamination=0.1) 
model.fit(X_train_scaled)

,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",100
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'
,"contamination contamination: 'auto' or float, default='auto'The amount of contamination of the data set, i.e. the proportionof outliers in the data set. Used when fitting to define the thresholdon the scores of the samples.- If 'auto', the threshold is determined as in the original paper.- If float, the contamination should be in the range (0, 0.5]... versionchanged:: 0.22 The default value of ``contamination`` changed from 0.1 to ``'auto'``.",0.1
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator.- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.Note: using a float number less than 1.0 or integer less than number offeatures will enable feature subsampling and leads to a longer runtime.",1.0
,"bootstrap bootstrap: bool, default=FalseIf True, individual trees are fit on random subsets of the trainingdata sampled with replacement. If False, sampling without replacementis performed.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for :meth:`fit`. ``None`` means 1unless in a :obj:`joblib.parallel_backend` context. ``-1`` means usingall processors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo-randomness of the selection of the featureand split values for each branching step and each tree in the forest.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42
,"verbose verbose: int, default=0Controls the verbosity of the tree building process.",0
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fit a wholenew forest. See :term:`the Glossary `... versionadded:: 0.21",False


In [9]:
import joblib

joblib.dump(model, "../../models/anomaly_model.pk")
joblib.dump(scaler, "../../models/scaler.pk")

['../../models/scaler.pk']

In [10]:
model.predict(X_test_scaled)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1, -1, -1,  1,
        1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1])

In [13]:
model.decision_function(X_train_scaled)

array([ 0.03441181,  0.02798689,  0.04924583,  0.09644309,  0.10580499,
        0.06979958,  0.12329442,  0.13697482,  0.13619679, -0.00998818,
        0.04296043, -0.09524843,  0.08166067,  0.04731142, -0.01027259,
        0.1367898 ,  0.0997253 ,  0.00209917,  0.06279215,  0.02650355,
        0.05873096,  0.08164966, -0.04307984,  0.07298965,  0.11596153,
        0.08904155,  0.02727575,  0.13336812,  0.06800058,  0.14015637,
        0.05674359,  0.12383637,  0.01288077,  0.03852895,  0.06063718,
        0.07505129,  0.07275501,  0.04340214, -0.01637493,  0.09606785,
        0.08170858,  0.02617785,  0.06240659,  0.09600007,  0.01085232,
        0.09495553,  0.05831797,  0.11569567,  0.1260183 ,  0.11214748,
        0.01206425,  0.0535122 ,  0.04020369,  0.06922495, -0.07995259,
        0.11539839, -0.04871959,  0.08872543, -0.02704563,  0.10937931,
        0.07300585,  0.0991519 ,  0.024606  ,  0.03887596,  0.1131255 ,
        0.06705292,  0.11405884,  0.08732514, -0.0641381 ,  0.09

In [14]:
df_trained = X_train.copy()
df_trained['prediction'] = model.predict(X_train_scaled)
df_trained.head(10)

,amount,past_transaction_count,user_avg_amount,amount_minus_user_avg,time_gap_minutes,hour,day_of_week,prediction
249,16504,53,7941.415094,8562.584906,7085.0,9,0,1
433,18236,78,7176.782051,11059.217949,48.0,2,6,1
19,690,1,7043.000000,-6353.000000,3896.0,11,0,1
322,1087,64,6655.671875,-5568.671875,3034.0,11,0,1
332,482,70,7137.028571,-6655.028571,5563.0,10,4,1
56,701,13,6853.615385,-6152.615385,106.0,13,0,1
301,2027,58,6953.896552,-4926.896552,459.0,19,3,1
229,3845,47,7011.957447,-3166.957447,290.0,15,3,1
331,4321,66,7495.757576,-3174.757576,1030.0,11,3,1
132,24153,22,9414.136364,14738.863636,3888.0,1,3,-1


In [16]:
df_trained['prediction'].value_counts() # Contamination

prediction
 1    360
-1     40
Name: count, dtype: int64

In [17]:
anomalies = df_trained[df_trained['prediction'] == -1]
anomalies

,amount,past_transaction_count,user_avg_amount,amount_minus_user_avg,time_gap_minutes,hour,day_of_week,prediction
132,24153,22,9414.136364,14738.863636,3888.0,1,3,-1
423,44887,84,8438.166667,36448.833333,5866.0,3,4,-1
464,28112,93,8080.419355,20031.580645,1238.0,1,4,-1
320,1916,62,7892.983871,-5976.983871,18443.0,20,6,-1
17,4283,3,1998.666667,2284.333333,2749.0,19,6,-1
462,46749,92,7660.108696,39088.891304,2971.0,4,3,-1
3,2080,0,0.000000,2080.000000,0.0,21,2,-1
405,37340,87,7163.068966,30176.931034,2003.0,1,3,-1
453,38442,96,7664.958333,30777.041667,1859.0,4,0,-1
37,33435,8,6087.000000,27348.000000,6191.0,4,2,-1
